In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def load_data(path = r"E:\Documents\Data\Datasets"):
    # point to the data set directory and choose the file to load
    import os
    p = path
    os.chdir(p) 

    #choose a data set
    while True:
        files = []
        with os.scandir(p) as dir:
            for count,entry in enumerate(dir):
                print(f"{count}) {entry.name}")
                files.append(entry.name)




        data = int(input('Enter the file index: '))
        data = os.path.join(p,files[data])
        if not data.endswith('.csv'):
            p = data
            print("\nThis is a directory\n")
        else:
            print(f'\nUsing the {os.path.basename(data)} dataset')
            break
            
    return data

In [2]:
dataset = load_data()

0) deepnlp
1) fake-and-real-news-dataset
2) game-of-thrones-srt
3) graduate-admissions
4) news-headlines-dataset-for-sarcasm-detection
5) ted-talks
6) young-people-survey
7) developer_survey_2019
8) Video 2020-04-17_2020-05-15 iCburks
9) Video 2013-04-06_2020-05-15 Gabe Flomo
10) Yelp Data
11) Udemy
12) NLP Data
13) data-police-shootings-master
14) emnist digits
15) CSV
16) Heart attack
17) Student placement
18) Background checks
19) Myers Briggs Type Indicator
20) Twitter US Airline sentiment
21) reddit sarcasm
22) developer survey 2017
23) twitter threads
24) Alchohol consumption
25) Fake Insta account
26) youtube data
27) IKEA Furniture
28) question classifcation
29) question answering
30) star data
31) Credit risk
32) tweets analysis
33) New york times comments
34) bank marketing campaings
35) Python questions from S.O
Enter the file index: 15

This is a directory

0) winequality-red.csv
1) AB_NYC_2019.csv
2) ArXiv_old.csv
3) books.csv
4) cannabis.csv
5) diamonds.csv
6) ETH_1h.csv.

In [3]:
df = pd.read_csv(dataset)

In [14]:
print(df.shape)
df.head()

(1000, 8)


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [21]:
# see the categorical class types and the value counts for each
categories = [col for col in df.columns if df[col].dtype == 'O']
print('Distribution of categorical variables\n')
for category in categories:
    print(df[category].value_counts(normalize = True)*100)
    print()

Distribution of categorical variables

female    51.8
male      48.2
Name: gender, dtype: float64

group C    31.9
group D    26.2
group B    19.0
group E    14.0
group A     8.9
Name: race/ethnicity, dtype: float64

some college          22.6
associate's degree    22.2
high school           19.6
some high school      17.9
bachelor's degree     11.8
master's degree        5.9
Name: parental level of education, dtype: float64

standard        64.5
free/reduced    35.5
Name: lunch, dtype: float64

none         64.2
completed    35.8
Name: test preparation course, dtype: float64



In [27]:
# add a new column for the overall score
df['overall score'] = df[['math score','reading score','writing score']].apply(lambda x,y,z: (x+y+z)/3)

TypeError: ("<lambda>() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index math score')

In [31]:
df[['math score','reading score','writing score']].apply(lambda x: x+x+x/3)

,math score,reading score,writing score
0,168.000000,168.000000,172.666667
1,161.000000,210.000000,205.333333
2,210.000000,221.666667,217.000000
3,109.666667,133.000000,102.666667
4,177.333333,182.000000,175.000000
...,...,...,...
995,205.333333,231.000000,221.666667
996,144.666667,128.333333,128.333333
997,137.666667,165.666667,151.666667
998,158.666667,182.000000,179.666667
